## Problem Statement

### Context

AllLife Bank is a US bank that has a growing customer base. The majority of these customers are liability customers (depositors) with varying sizes of deposits. The number of customers who are also borrowers (asset customers) is quite small, and the bank is interested in expanding this base rapidly to bring in more loan business and in the process, earn more through the interest on loans. In particular, the management wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors).

A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise campaigns with better target marketing to increase the success ratio.

You as a Data scientist at AllLife bank have to build a model that will help the marketing department to identify the potential customers who have a higher probability of purchasing the loan.

### Objective

To predict whether a liability customer will buy personal loans, to understand which customer attributes are most significant in driving purchases, and identify which segment of customers to target more.

### Data Dictionary
* `ID`: Customer ID
* `Age`: Customer’s age in completed years
* `Experience`: #years of professional experience
* `Income`: Annual income of the customer (in thousand dollars)
* `ZIP Code`: Home Address ZIP code.
* `Family`: the Family size of the customer
* `CCAvg`: Average spending on credit cards per month (in thousand dollars)
* `Education`: Education Level. 1: Undergrad; 2: Graduate;3: Advanced/Professional
* `Mortgage`: Value of house mortgage if any. (in thousand dollars)
* `Personal_Loan`: Did this customer accept the personal loan offered in the last campaign? (0: No, 1: Yes)
* `Securities_Account`: Does the customer have securities account with the bank? (0: No, 1: Yes)
* `CD_Account`: Does the customer have a certificate of deposit (CD) account with the bank? (0: No, 1: Yes)
* `Online`: Do customers use internet banking facilities? (0: No, 1: Yes)
* `CreditCard`: Does the customer use a credit card issued by any other Bank (excluding All life Bank)? (0: No, 1: Yes)

## Importing necessary libraries

In [ ]:
# Installing the libraries with the specified version.
#!pip install numpy==1.25.2 pandas==1.5.3 matplotlib==3.7.1 seaborn==0.13.1 scikit-learn==1.2.2 sklearn-pandas==2.2.0 -q --user

**Note**:

1. After running the above cell, kindly restart the notebook kernel (for Jupyter Notebook) or runtime (for Google Colab), write the relevant code for the project from the next cell, and run all cells sequentially from the next cell.

2. On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [2]:
# Import required libraries

# Data Handling
import pandas as pd  
import numpy as np  

# Visualization
import matplotlib.pyplot as plt  
import seaborn as sns  

# Machine Learning - Model Selection & Preprocessing
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler, LabelEncoder  

# Machine Learning - Classification Models
from sklearn.ensemble import RandomForestClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.linear_model import LogisticRegression  

# Evaluation Metrics
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, 
    f1_score, recall_score, precision_score, 
    roc_auc_score, roc_curve
)

## Loading the dataset

In [3]:
# Load the dataset
Loan_df = pd.read_csv("Loan_Modelling.csv")

In [4]:
#create a copy of the dataset to avoid modifying the original data
Loan_Copy_df = Loan_df.copy()
Loan_Copy_df

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


## Data Overview

* Observations
* Sanity checks

I am doing the sanity checks first. Some of the things that I will do are 
1. Data Dimensions and Types
2. Missing Values/Duplicated values or non-null values
3. Constant or Near-Constant columns
4. Outlier Inspections using plots. This can be done continuous numerical variables like Age, Experience, Income, CCAvg, Mortgage.

Showing the shape of the data set. 
From the results below the dataset has 5000 rows and 14 columns. A moderate-sized dataset which is large enough for training meaningful models without performance issues. 

In [7]:
#show the shape of the dataset
Loan_Copy_df.shape
print(f"The dataset has {Loan_Copy_df.shape[0]} rows and {Loan_Copy_df.shape[1]} columns.")

The dataset has 5000 rows and 14 columns.


In [13]:
#display the data types of the columns
Loan_Copy_df.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIPCode                 int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal_Loan           int64
Securities_Account      int64
CD_Account              int64
Online                  int64
CreditCard              int64
dtype: object

In [14]:
Loan_Copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIPCode             5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal_Loan       5000 non-null   int64  
 10  Securities_Account  5000 non-null   int64  
 11  CD_Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [10]:
#Show the first few rows of the dataset
Loan_Copy_df.head()

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [11]:
#show the last few rows of the dataset
Loan_Copy_df.tail()

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0
4999,5000,28,4,83,92612,3,0.8,1,0,0,0,0,1,1


In [15]:
# Check for missing values in Loan_Copy_df (not the original)
Loan_Copy_df.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIPCode               0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal_Loan         0
Securities_Account    0
CD_Account            0
Online                0
CreditCard            0
dtype: int64

In [17]:
duplicate_Count = Loan_Copy_df.duplicated().sum()
if duplicate_Count > 0:
    print("The dataset has", duplicate_Count, "duplicate rows.")
else:
    print("The dataset has no duplicate rows.")

The dataset has no duplicate rows.


In [28]:
# Compute unique value counts
unique_values = Loan_Copy_df.nunique().reset_index()
unique_values.columns = ['Column', 'Unique Values']

# Display as a formatted table
print("\nUnique Values in Each Column:")
unique_values


Unique Values in Each Column:


,Column,Unique Values
0,ID,5000
1,Age,45
2,Experience,47
3,Income,162
4,ZIPCode,467
5,Family,4
6,CCAvg,108
7,Education,3
8,Mortgage,347
9,Personal_Loan,2


In [22]:
Loan_Copy_df.describe(include='all')

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93169.257000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,1759.455086,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,90005.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [32]:
Loan_Copy_df.drop(columns=['ID'],axis=1, inplace=True)  # Drop the 'Loan_ID' column as it is not needed for analysis

In [33]:
Loan_Copy_df

,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


##Overall observations from sanity check and the shape of the dataset

- 5000 Customers 
- Small number of features allowing us to perform thorough EDA and feature importance tracking
- Some categorical variables
- Personal Loan is the target variable. This affects the metric choice like ROC-AUC, precision recall

## Column level obeservations and Processing

In [36]:
Loan_Copy_df['Personal_Loan'].value_counts()

0    4520
1     480
Name: Personal_Loan, dtype: int64

In [37]:
Loan_Copy_df['Personal_Loan'].value_counts(normalize=True) * 100


0    90.4
1     9.6
Name: Personal_Loan, dtype: float64

In [ ]:
family_loan_pct = pd.crosstab(
    Loan_Copy_df['Family'],
    Loan_Copy_df['Personal_Loan'],
    normalize='index'
) * 100

# Optional: round to 2 decimal places
family_loan_pct = family_loan_pct.round(2)
family_loan_pct.columns = ['Did Not accept the Loan (%)', 'Accepted the Loan (%)']

family_loan_pct

,Did Not Take Loan (%),Accepted Loan (%)
Family,,
1,92.73,7.27
2,91.82,8.18
3,86.83,13.17
4,89.03,10.97


In [39]:
pd.qcut(Loan_Copy_df['Income'], 5)  # Income quintiles

0        (33.0, 52.0]
1        (33.0, 52.0]
2       (7.999, 33.0]
3       (78.0, 113.0]
4        (33.0, 52.0]
            ...      
4995     (33.0, 52.0]
4996    (7.999, 33.0]
4997    (7.999, 33.0]
4998     (33.0, 52.0]
4999    (78.0, 113.0]
Name: Income, Length: 5000, dtype: category
Categories (5, interval[float64, right]): [(7.999, 33.0] < (33.0, 52.0] < (52.0, 78.0] < (78.0, 113.0] < (113.0, 224.0]]

In [35]:
Loan_Copy_df['ZIP_Prefix'] = Loan_Copy_df['ZIPCode'].astype(str).str[:2]
# Display the first few rows to verify the new column
Loan_Copy_df.head()

,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,ZIP_Prefix
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,91
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,90
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,94
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,94
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,91


## Exploratory Data Analysis.

- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

**Questions**:

1. What is the distribution of mortgage attribute? Are there any noticeable patterns or outliers in the distribution?
2. How many customers have credit cards?
3. What are the attributes that have a strong correlation with the target attribute (personal loan)?
4. How does a customer's interest in purchasing a loan vary with their age?
5. How does a customer's interest in purchasing a loan vary with their education?

## Data Preprocessing

* Missing value treatment
* Feature engineering (if needed)
* Outlier detection and treatment (if needed)
* Preparing data for modeling
* Any other preprocessing steps (if needed)

## Model Building

### Model Evaluation Criterion

*


### Model Building

## Model Performance Improvement

## Model Performance Comparison and Final Model Selection

## Actionable Insights and Business Recommendations


* What recommedations would you suggest to the bank?

___